In [36]:
import logging
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd

# In Jupyter, __file__ is not defined, so use the current working directory
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
# from frankenstein.graph import FrankensteinGraph
import matcher

# Suppress all logging from FrankensteinGraph and its dependencies
logging.getLogger().setLevel(logging.ERROR)

# Call a pointless matplotlib function to stop it being autoremoved by Jupyter
plt.figure(figsize=(10, 6))

# df = pd.read_json('../eval/runs/gpt-4o-mini_answerable-full.jsonl', orient='records', lines=True)
df = pd.read_json('../eval/runs/gpt-4.1-mini_answerable-full.jsonl', orient='records', lines=True)


m = matcher.Matcher()

<Figure size 1000x600 with 0 Axes>

In [37]:
df['correct'] = df['correct'].astype(bool)
df.head()

,id,question_template,question,actions,answer,slot_values,answerable,data_availability,answer_format,messages,pred,correct,error
0,d850e2c9-0254-4152-83ae-c8163d968484,RegionPropertyRatio,What was the ratio of the highest value to the...,"[{'name': 'get_country_codes_in_region', 'argu...",1.944958,"{'region': 'Eastern Europe', 'property': 'PA.N...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",The ratio of the highest value to the lowest f...,True,0
1,286344d6-803c-4dff-94ae-43d1b7623d70,RegionProportion,What proportion of the total tuberculosis inci...,"[{'name': 'get_country_code_from_name', 'argum...",0.176755,"{'property': 'SH.TBS.INCD', 'subject': 'KAZ', ...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",The proportion of the total tuberculosis incid...,True,100
2,36479e92-e9cd-46f9-9c2f-34f7f83305c5,IncreasePropertyComparison,Which country in Melanesia had the lowest incr...,"[{'name': 'get_country_codes_in_region', 'argu...",[Papua New Guinea],"{'region': 'Melanesia', 'operator': 'lowest', ...",True,full,list[str],"[{'role': 'system', 'content': 'You are a help...",Papua New Guinea,True,0
3,fb7e49e8-28ab-46a6-a6f1-815f35343331,PropertyOfSubject,What was the crude birth rate of Bahrain in 2009?,"[{'name': 'get_country_code_from_name', 'argum...",15.462,"{'property': 'SP.DYN.CBRT.IN', 'subject': 'BHR...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",The crude birth rate of Bahrain in 2009 was 15...,True,0
4,cd6deec0-ecde-4e63-9756-d64a5b90a066,CountryPropertyComparison,"In 2010, was the annual population increase pe...","[{'name': 'get_country_code_from_name', 'argum...",True,"{'subject_a': 'CIV', 'property': 'SP.POP.GROW'...",True,full,bool,"[{'role': 'system', 'content': 'You are a help...","Yes, in 2010, the annual population increase p...",True,0


In [46]:
df['correct'].groupby(df['question_template']).value_counts()

question_template           correct
AverageChange               True       1
CountryPropertyComparison   True       2
FactorIncreaseComparison    True       2
IncreasePropertyComparison  True       1
PropertyOfSubject           True       2
PropertyRatioComparison     True       3
RegionComparison            True       2
RegionComparisonResult      True       2
RegionPropertyRatio         True       1
RegionProportion            True       2
RegionProportionChange      True       1
TotalProperty               True       1
Name: count, dtype: int64